# 1.  Single object diffusion demo

This notebook allows for experimentation with object detection + segmentation --> diffusion.

To use it first:

- select a gpu runtime: make sure you have chosen a GPU runtime from the options above - choose A100 if available.
- activate the first few cells to 
  - perform a machine check: make sure your runtime is employing a GPU with sufficient memory
  - download associated files and installs
- start experimenting!

## 1.  machine setup and installs

### pre-launch machine check

Activate the code block below to double check that you are using a GPU runtime for the experiments.  You don't have to use one if you can't, but the experiments will run significantly faster (in particular the diffusion steps) if you are using one.

In [ ]:
# check for GPU runtime
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# check for memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

### installs

Activate the cell below to install all necessary packages to your collab enviroment for these experiments.

In [ ]:
# install reqiured libraries 
!pip install "ultralytics==8.0.111" "transformers==4.29.2" "timm==0.9.2" "diffusers==0.16.1" "safetensors==0.3.1" "accelerate==0.19.0"

## 2. start experimenting!

In [ ]:
# import segmentation and diffusion modules
# the first time you activate this cell models will be downloaded to your machine
from segmenter import segment_img, label_lookup_dict
from diffuser import diffuse_segmented_img

In [ ]:
# Example usage - picture of bald man holding can
url = "https://github.com/jermwatt/morphi_lab/blob/collab_demos/test_data/test_input/test_img.png?raw=true"
output_path = "/content/test_img.png"
download_file(url, output_path)

In [ ]:
# man holding donut - we'll use this one for testing
url = "https://www.shutterstock.com/image-photo/surprised-young-man-holding-donut-260nw-586330142.jpg"
output_path = "/content/test_donut.png"
download_file(url, output_path)

## 1.4.  segmentation

In [ ]:
img_path = "/content/test_donut.png"
labels = ['bottle','cup']
labels = ['person']
conf = 0.05
max_det = 1
img, mask, seg = segment_image(img_path,
                               labels=labels,
                               conf=conf,
                               max_det=max_det)

In [ ]:
seg.show_result()

## 1.5. diffuse object

In [ ]:
diffused_img = diffuse(img,
                       mask,
                       prompt='an ape, smiling, high resolution, holding something',
                       seed=3433)

## 1.6. Prep all imgs for showing 

In [ ]:
plot_diffuse_results(img,
                     mask,
                     diffused_img)